<a href="https://colab.research.google.com/github/chhezue/6th-Seoul-Edu-BigData/blob/main/Lecture_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import os
import re
import time
import warnings
import datetime
import pickle
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
lec_off_df = pd.read_csv('/content/seoul-offline.csv', encoding = 'cp949')
lec_on_df  = pd.read_csv('/content/seoul-online.csv', encoding = 'cp949')

In [28]:
lec_off_df.head()

,사이트 아이디,ASP 아이디,강의 아이디,강의 구분,강의 명,수강신청 시작 일자,수강신청 종료 일자,강의 시작 일자,강의 종료 일자,강의대상,정원,수강신청 URL,등록 일자,수정 일자,교육기관 고유아이디,교육기관명,교육기관 자치구,교육기관 위도 좌표,교육기관 경도 좌표
0,lc1,ASP00001,ASP00001Z401120240000082,오프라인,영어로배우는 키성장요가,202406100000,202407020000,20240702,20240820,NaN,14,https://www.seoullifelong.co.kr/index.php?modu...,20240610111046,20240610111046,Z4011,서울관악복합평생교육센터,관악구,37.472611,126.957554
1,lc1,ASP00001,ASP00001Z401120240000084,오프라인,창의쑥쑥키즈페인팅,202406100000,202407100000,20240710,20240731,NaN,15,https://www.seoullifelong.co.kr/index.php?modu...,20240610111046,20240610111046,Z4011,서울관악복합평생교육센터,관악구,37.472611,126.957554
2,lc1,ASP00001,ASP00001Z401120240000083,오프라인,건강튼튼 어린이발레,202406100000,202407080000,20240708,20240823,NaN,12,https://www.seoullifelong.co.kr/index.php?modu...,20240610111046,20240610111046,Z4011,서울관악복합평생교육센터,관악구,37.472611,126.957554
3,lc1,ASP00001,ASP00001Z401120240000085,오프라인,팝송으로 배우는 잉글리쉬,202406100000,202407020000,20240702,20240820,NaN,15,https://www.seoullifelong.co.kr/index.php?modu...,20240610111046,20240610111046,Z4011,서울관악복합평생교육센터,관악구,37.472611,126.957554
4,lc1,ASP00001,ASP00001Z401120240000078,오프라인,시니어 모델워킹,202406100000,202407010000,20240701,20240724,NaN,25,https://www.seoullifelong.co.kr/index.php?modu...,20240610111045,20240610111045,Z4011,서울관악복합평생교육센터,관악구,37.472611,126.957554


In [29]:
lec_on_df.columns

Index(['아이디', '강의 아이디', '강의 카테고리 아이디', '강의 카테고리명', '강의 명', '비용', '강의 구분',
       '주차 사용 여부', '분반 일련번호', '수강신청 시작 일자', '강의 시작 일자', '등록 일자',
       '수강신청 시작 일자.1', '수료증 로트 명', '기관 아이디', '기관명', '기관 구', 'X좌표', 'Y좌표',
       '요청여부', '인기여부', '신규여부', '상태', '강의 이미지 파일 경로', '수집데이터 아이디', '과목 아이디',
       '미리보기 주소', '콘텐츠 아이디', '회차번호', '회차수'],
      dtype='object')

In [30]:
lec_on_df.head()

,아이디,강의 아이디,강의 카테고리 아이디,강의 카테고리명,강의 명,비용,강의 구분,주차 사용 여부,분반 일련번호,수강신청 시작 일자,...,인기여부,신규여부,상태,강의 이미지 파일 경로,수집데이터 아이디,과목 아이디,미리보기 주소,콘텐츠 아이디,회차번호,회차수
0,ASP00001,ASP00001S100120240107312,202303131239826,인권,"기후위기와 인권, 그리고 새로운 사회로 전환",무료,1,N,1,2024.06.04~2024.12.31,...,N,N,ING,/upload/course/ASP00001/2024/06/썸네일(기후위기와 인권)-...,NaN,S12330,https://cdn.seoul.go.kr:443/mvod/_definst_/mp4...,202406031418241717391904759,131892,1
1,ASP00001,ASP00001S100120240107309,202303136122092,사회/교양,2024년 서울시 온라인 예산학교,무료,1,N,1,2024.06.01~2024.12.31,...,N,N,ING,/upload/course/ASP00001/2024/05/붙임3. 서울시 온라인 예...,NaN,S12329,https://cdn.seoul.go.kr:443/mvod/_definst_/mp4...,202405300922101717028530137,131886,1
2,ASP00001,ASP00001S100120240107300,202303136122092,사회/교양,"두 마리 토끼를 잡아라, 서울시 일?생활 균형 온라인 교육",무료,1,N,1,2024.05.21~2024.12.31,...,N,N,ING,/upload/course/ASP00001/2024/05/썸네일 이미지[4].jpg,NaN,S12328,https://cdn.seoul.go.kr:443/mvod/_definst_/mp4...,202405210908101716250090783,131880,1
3,ASP00001,ASP00001S100120240107290,202303095029768,직업상담사,직업상담사 필기 - 노동관계법규,무료,1,N,1,2024.05.22~2024.12.31,...,N,N,ING,/upload/course/ASP00001/2024/thumbnails/ASP000...,NaN,S12324,https://cdn.seoul.go.kr:443/mvod/_definst_/mp4...,202405201528271716186507758,131809,1
4,ASP00001,ASP00001S100120240107291,202303095029768,직업상담사,직업상담사 필기 - 노동시장론,무료,1,N,1,2024.05.22~2024.12.31,...,N,N,ING,/upload/course/ASP00001/2024/thumbnails/ASP000...,NaN,S12325,https://cdn.seoul.go.kr:443/mvod/_definst_/mp4...,202405201535201716186920175,131836,1


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# 텍스트 벡터화 및 모델 학습
tfidf = TfidfVectorizer()
X_on = tfidf.fit_transform(lec_on_df['강의 명'])
y_on = lec_on_df['강의 카테고리명']

# 레이블 인코딩
le = LabelEncoder()
y_on_encoded = le.fit_transform(y_on)

# 모델 학습
model = LogisticRegression(random_state=42)
model.fit(X_on, y_on_encoded)

# 오프라인 강의 데이터에 대해 예측
X_off = tfidf.transform(lec_off_df['강의 명'])
y_off_pred_encoded = model.predict(X_off)
y_off_pred = le.inverse_transform(y_off_pred_encoded)

# 예측된 카테고리명 열 추가
lec_off_df['강의 카테고리명'] = y_off_pred

# 병합
merged_df = pd.merge(lec_off_df, lec_on_df, left_on=['강의 명', '강의 카테고리명'], right_on=['강의 명', '강의 카테고리명'], how='outer')

# 강의명과 강의 카테고리명 출력
output_3_df = merged_df[['강의 명', '강의 카테고리명']]
print(output_3_df)

                     강의 명 강의 카테고리명
0            영어로배우는 키성장요가    사회/교양
1               창의쑥쑥키즈페인팅    사회/교양
2              건강튼튼 어린이발레    사회/교양
3           팝송으로 배우는 잉글리쉬    사회/교양
4                시니어 모델워킹    사회/교양
...                   ...      ...
1393         개인정보보호편(노년층)       기타
1394        개인정보보호편(청소년층)       기타
1395  공공데이터로 만들어가는 새로운 세상    사회/교양
1396          e-직장인의 건강관리     건강관리
1397   함께하는 부모 학습_드디어 부모!     부모교육

[1398 rows x 2 columns]


In [32]:
# 구글 드라이브 내 경로 설정
file_path_3 = '/content/merged_lecture.csv'

# 데이터프레임을 CSV 파일로 저장
output_3_df.to_csv(file_path_3, index=False)

In [33]:
output_3_df['강의 카테고리명'].unique()

array(['사회/교양', '콘텐츠산업 전문과정', '기타', '영어', '국가기술자격', '사회복지사', '웹툰/이모티콘',
       '직업상담사', '창업과정', '공인중개사 연수교육', '인권', '전문기술과정', '셀프브랜딩', '리더십/인사조직',
       '공인중개사', '주택관리사', '인문학', '직무역량과정', '방송/영상', '부모교육', '금융/회계',
       '경영일반', 'OA활용', '일본어', '문화예술', '기타외국어', '신기술교육', '건강관리', '중국어',
       '프로그래밍', '아동/청소년', '금융', '창업', '인공지능', 'AR/VR', '취업', '시니어취업',
       '취미생활', '멀티미디어', '국가공인민간자격', '소셜미디어'], dtype=object)

In [34]:
# 텍스트 전처리 함수
okt = Okt()

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z0-9가-힣\s]', '', text)  # 특수문자 제거
    text = okt.morphs(text)  # 형태소 분석
    return ' '.join(text)

In [35]:
# 전처리 시간 측정
start_time = time.time()
output_3_df['강의 명'] = output_3_df['강의 명'].apply(preprocess_text)
output_3_df['강의 카테고리명'] = output_3_df['강의 카테고리명'].apply(preprocess_text)
preprocessing_time = time.time() - start_time
print(f"전처리 시간: {preprocessing_time:.2f}초")

전처리 시간: 2.69초
